In [10]:
# connect mysql
import pymysql
conn = pymysql.connect(host = 'localhost', user='ys9925', password='1Q2w3e4r!@', db='crime_prediction', charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

In [17]:
# select from Dataset - Choose scenario

# Season
num = int(input('1.Spring 2.Summer 3.Fall 4.Winter '))
if num == 1: season = 'Spring'
elif num == 2: season = "Summer" 
elif num == 3: season = "Fall" 
else: season = "Winter"

# Time    
num = int(input("1.Morning 2.Afternoon 3.Evening 4.Midnight "))
if num == 1: time = "Morning"  
elif num == 2: time = "Afternoon" 
elif num == 3: time = "Evening" 
else: time = "Midnight"
    
# Weather    
num = int(input("1.눈/비 2.맑음 3.흐림 "))
if num == 1: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
elif num == 2: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
else: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 

dataset_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Dataset ' + where
grid_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Grid WHERE time = \''+ time + '\''
curs.execute(dataset_sql)
dataset_result = curs.fetchall()
curs.execute(grid_sql)
grid_result = curs.fetchall()

1.Spring 2.Summer 3.Fall 4.Winter 4
1.Morning 2.Afternoon 3.Evening 4.Midnight 3
1.눈/비 2.맑음 3.흐림 3


In [18]:
# Transform to Pandas Dataframe
import pandas as pd
dataset_result = pd.DataFrame(dataset_result)
grid_result = pd.DataFrame(grid_result)
result = pd.concat([dataset_result, grid_result])
result['crime_num'] = result.groupby(['grid']).grid.transform('count') -1
result = result.drop_duplicates()
result = result.sort_values('grid')
result

,grid,CCTV_num,police_value,resident,floating,crime_num
118,0,0,0,20,930,1
294,1,0,0,30,670,1
2,2,0,0,20,810,0
308,3,0,0,20,660,1
137,4,0,0,40,760,3
...,...,...,...,...,...,...
361,1102,8,0,310,6430,1
586,1103,0,0,330,6250,2
709,1104,0,0,450,6400,1
521,1105,0,0,240,6770,2


In [19]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # DecisionTree 모듈, 시각화 모듈
from sklearn.model_selection import train_test_split  # train, test dataset 분류
from sklearn.model_selection import GridSearchCV # cross validation, hyper parameter
from collections import Counter # 라벨 분포 확인
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import sys
import pydot
np.set_printoptions(threshold=sys.maxsize)
# Decision tree Classifier 생성

# 학습해야하는 변수
data = result[["CCTV_num", "police_value", "resident", "floating"]].to_numpy()
# 맞춰야하는 변수: crime_num
target = result.crime_num.to_numpy()

# dataset 분할
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=10)

# 하이퍼 파라미터 찾기
tree = DecisionTreeClassifier()
parameters = {'max_depth' : [x for x in range(3, 31, 1)], 'min_samples_leaf' : [x for x in range(1, 30, 1)], 'min_samples_split' : [x for x in range(2, 30, 1)]}

# param_grid의  하이퍼 파라미터를 5개의 train, test set fold로 나누어 테스트 수행 설정
## refit=True가 기본 설정으로 True이면 가장 좋은 파라미터 설정으로 재학습 시킴
grid_dtree = GridSearchCV(tree, param_grid=parameters, cv=5, refit=True)

#학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습, 평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 데이터 프레임으로 반환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.536723,517,0.508475,0.542373,0.514124
1,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.536723,517,0.508475,0.542373,0.514124
2,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.536723,517,0.508475,0.542373,0.514124
3,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.536723,517,0.508475,0.542373,0.514124
4,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.536723,517,0.508475,0.542373,0.514124
...,...,...,...,...,...,...
22731,"{'max_depth': 30, 'min_samples_leaf': 29, 'min...",0.516384,2438,0.480226,0.548023,0.531073
22732,"{'max_depth': 30, 'min_samples_leaf': 29, 'min...",0.516384,2438,0.480226,0.548023,0.531073
22733,"{'max_depth': 30, 'min_samples_leaf': 29, 'min...",0.516384,2438,0.480226,0.548023,0.531073
22734,"{'max_depth': 30, 'min_samples_leaf': 29, 'min...",0.516384,2438,0.480226,0.548023,0.531073


In [20]:

print('GridSearch 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearch 최고 점수: ', grid_dtree.best_score_)

# GridSearchCV의 refit으로 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estmator_ 는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print('테스트 데이터세트 정확도: {0: .4f}'.format(accuracy_score(y_test, pred)))

# predic test_set labels
y_pred = estimator.predict(data)
# Compute test-set RMS
mse_dt = MSE(target, y_pred)
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Grid RMSE: ", rmse_dt, '\n')
print(y_pred)

# 시각화를 위해 학습에 사용한 feature 라벨 추출
feature_names = result.columns.tolist()
feature_names = feature_names[1:5]
# 트리 시각화
dot_data = export_graphviz(estimator, out_file="1_1_1_DecisionTree.dot", feature_names = feature_names, filled=True, rounded=True, special_characters=True)
# 인코딩
(graph,) = pydot.graph_from_dot_file("1_1_1_DecisionTree.dot", encoding='utf8')
# png로 저장
graph.write_png("1_1_1_DecisionClass.png") 

# 전체 Grid의 범죄 발생 확률 구하기
total = np.sum(y_pred)
result_prob = y_pred / total
#print(result_prob)


GridSearch 최적 파라미터:  {'max_depth': 3, 'min_samples_leaf': 20, 'min_samples_split': 2}
GridSearch 최고 점수:  0.5412429378531073
테스트 데이터세트 정확도:  0.5721
Grid RMSE:  1.0027063648947991 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(target, y_pred)

target_unique = np.unique(target)
confusion_df = pd.DataFrame(index=range(0), columns=['value', 'True', 'False'])

for i in range(len(target_unique)):
    true = 0
    false = 0
    for j in range(len(target_unique)):
        if abs(j - i) < 3:
            true += confusion_matrix[i][j]
        else:
            false += confusion_matrix[i][j]
            
    row = [target_unique[i], true, false]
    confusion_df = confusion_df.append(pd.Series(row, index=confusion_df.columns), ignore_index=True)
    
true_total = confusion_df['True'].sum()
false_total = confusion_df['False'].sum()

print("정확도 : " + str(true_total / (true_total + false_total)))
 
confusion_df

정확도 : 0.9683830171635049


,value,True,False
0,0,600,0
1,1,347,0
2,2,101,0
3,3,24,16
4,4,0,16
5,5,0,3
